In [ ]:
# Importing the libraries
import numpy as np
#import matplotlib.pyplot as plt

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout # GRU, Bidirectional
#from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error

In [4]:
data = pd.read_csv('/content/drive/MyDrive/files2/IBM_2006-01-01_to_2018-01-01.csv', index_col='Date', parse_dates=['Date'])
data.head() #view first five columns

Open   High    Low  Close    Volume Name
Date                                                 
03-01-2006  82.45  82.55  80.81  82.06  11715200  IBM
04-01-2006  82.20  82.50  81.33  81.95   9840600  IBM
05-01-2006  81.40  82.90  81.00  82.50   7213500  IBM
06-01-2006  83.95  85.03  83.41  84.95   8197400  IBM
09-01-2006  84.10  84.25  83.38  83.73   6858200  IBM

In [7]:
# Convert '2016' and '2017' to datetime objects for slicing
# Assuming 'Date' is a datetime column
mytrain = data[data.index < '2017-01-01'].iloc[:, 1:2].values
mytest = data[data.index >= '2017-01-01'].iloc[:, 1:2].values

In [8]:
# Scaling the training set
sc = MinMaxScaler(feature_range=(0,1))  #MinMaxScaler(feature_range= (start,stop))
mytrain_scaled = sc.fit_transform(mytrain)  #instance.fit_transform(data)

In [9]:
mytrain_scaled #view the scaled values!
#82.55 ==> 0.06065...

array([[0.06065089],
       [0.06029868],
       [0.06311637],
       ...,
       [0.56522964],
       [0.5651592 ],
       [0.56318681]])

In [10]:
len(mytrain_scaled)

1992

In [11]:
I_train = []
O_train = []
for i in range(60, len(mytrain_scaled)):  # Change the upper bound of the range
    I_train.append(mytrain_scaled[i-60:i,0])
    O_train.append(mytrain_scaled[i,0])

In [12]:
I_train[0]

array([0.06065089, 0.06029868, 0.06311637, 0.0781206 , 0.07262609,
       0.07171034, 0.07657087, 0.07058326, 0.0669907 , 0.06494787,
       0.075796  , 0.07361229, 0.06417301, 0.05846717, 0.05388842,
       0.04811214, 0.04233587, 0.04402649, 0.0490279 , 0.04832347,
       0.05297267, 0.05614258, 0.05290223, 0.05325444, 0.04909834,
       0.04994365, 0.04839391, 0.04416737, 0.0485348 , 0.04719639,
       0.04825303, 0.05395886, 0.05663567, 0.05853762, 0.05959425,
       0.06375035, 0.06917442, 0.06889265, 0.06670893, 0.06910397,
       0.06797689, 0.0693153 , 0.07403494, 0.07311919, 0.06945618,
       0.06198929, 0.05719921, 0.05233869, 0.0610031 , 0.06325726,
       0.06755424, 0.07086503, 0.06276416, 0.06600451, 0.06198929,
       0.06586362, 0.06445478, 0.06621584, 0.06537053, 0.06586362])

In [13]:
I_train[0]

array([0.06065089, 0.06029868, 0.06311637, 0.0781206 , 0.07262609,
       0.07171034, 0.07657087, 0.07058326, 0.0669907 , 0.06494787,
       0.075796  , 0.07361229, 0.06417301, 0.05846717, 0.05388842,
       0.04811214, 0.04233587, 0.04402649, 0.0490279 , 0.04832347,
       0.05297267, 0.05614258, 0.05290223, 0.05325444, 0.04909834,
       0.04994365, 0.04839391, 0.04416737, 0.0485348 , 0.04719639,
       0.04825303, 0.05395886, 0.05663567, 0.05853762, 0.05959425,
       0.06375035, 0.06917442, 0.06889265, 0.06670893, 0.06910397,
       0.06797689, 0.0693153 , 0.07403494, 0.07311919, 0.06945618,
       0.06198929, 0.05719921, 0.05233869, 0.0610031 , 0.06325726,
       0.06755424, 0.07086503, 0.06276416, 0.06600451, 0.06198929,
       0.06586362, 0.06445478, 0.06621584, 0.06537053, 0.06586362])

In [14]:
I_train = np.array(I_train)   #converting into arrays!
O_train = np.array(O_train)

In [15]:
I_train = np.array(I_train)   #converting into arrays!
O_train = np.array(O_train)

In [16]:
I_train.shape #samples,steps

(1932, 60)

In [17]:
I_train = I_train.reshape(1932, 60, 1)
# Reshape to match the existing number of elements
print(I_train.shape)  # Verify the new shape

(1932, 60, 1)


In [18]:
I_train.shape

(1932, 60, 1)

In [19]:
#building the model

model = Sequential()
# First LSTM layer
model.add(LSTM(units=50, return_sequences=True, input_shape=(I_train.shape[1],1)))
model.add(Dropout(0.2))
# Second LSTM layer
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# Third LSTM layer
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# Fourth LSTM layer
model.add(LSTM(units=50))
model.add(Dropout(0.2))
# The output layer
model.add(Dense(units=1))

In [20]:
#compile
model.compile(optimizer='rmsprop',loss='mean_squared_error')

In [21]:
model.fit(I_train,O_train,epochs=50,batch_size=32)

Epoch 1/50
61/61 [==============================] - 21s 186ms/step - loss: 0.0267
Epoch 2/50
61/61 [==============================] - 9s 142ms/step - loss: 0.0117
Epoch 3/50
61/61 [==============================] - 10s 160ms/step - loss: 0.0100
Epoch 4/50
61/61 [==============================] - 8s 124ms/step - loss: 0.0094
Epoch 5/50
61/61 [==============================] - 9s 143ms/step - loss: 0.0081
Epoch 6/50
61/61 [==============================] - 10s 164ms/step - loss: 0.0069
Epoch 7/50
61/61 [==============================] - 14s 226ms/step - loss: 0.0068
Epoch 8/50
61/61 [==============================] - 11s 174ms/step - loss: 0.0061
Epoch 9/50
61/61 [==============================] - 7s 117ms/step - loss: 0.0054
Epoch 10/50
61/61 [==============================] - 9s 154ms/step - loss: 0.0053
Epoch 11/50
61/61 [==============================] - 8s 127ms/step - loss: 0.0050
Epoch 12/50
61/61 [==============================] - 9s 143ms/step - loss: 0.0052
Epoch 13/50
61/61 [=

In [22]:
model.save("StockIBM.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
#Test
from tensorflow.keras.models import load_model
import numpy as np
#load the model
model = load_model("/content/StockIBM.h5")
#60 values
user_input = [83.58746017, 80.34122742, 80.81285462, 82.00291895, 83.29159961, 81.44688209, 80.76357678, 83.80165525, 82.51120409, 82.48569342, 80.65383659, 81.69740582, 82.78338862, 83.37704145, 83.53829805, 80.20374715, 80.42476455, 82.69989097, 83.73417143, 82.95666579, 83.94381372, 82.37856671, 83.60787851, 81.80850007, 82.61239819, 82.06021048, 81.0673498, 83.7835985, 81.74964384, 83.10783276, 80.54146548, 80.49215929, 82.03237911, 81.79802128, 80.93914228, 83.90444224, 83.2143073, 81.31844381, 83.23807452, 83.4603476, 81.85039395, 81.3031393, 80.6351211, 83.65161771, 80.54893536, 82.22940581, 82.50289209, 80.59368723, 83.24620825, 81.77694404, 83.20275117, 83.43996357, 83.38335394, 82.69282925, 80.80822433, 80.30044039, 83.6842126, 82.82960781, 80.12425038, 82.25607533]

#2D array
user_input_array = np.array(user_input).reshape(60, 1)
#down-scale
sc = MinMaxScaler(feature_range=(0,1))
user_scaled = sc.fit_transform(user_input_array)
#3D array
user_scaled = user_scaled.reshape(1, 60, 1)
#predict
pred = model.predict(user_scaled)
print(pred) #down scale output
#up-scale
pred_original = sc.inverse_transform(pred)
print("The stock price is",pred_original[0][0]) #up scaled/original o/p


1/1 [==============================] - 3s 3s/step
[[0.50116783]]
The stock price is 82.03849


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
